In [127]:
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.initializers import Constant
from keras.layers import TextVectorization
import numpy as np
import pickle
import json

In [128]:
X = np.load('data/X.npy', allow_pickle=True)
y = np.load('data/y.npy')
embedding_matrix = np.load('data/embedding_matrix.npy')
num_tokens = np.load('data/num_tokens.npy').item()
embedding_dim = np.load('data/embedding_dim.npy').item()
max_tokens = np.load('data/max_tokens.npy').item()
max_len = np.load('data/max_len.npy').item()

In [129]:
vectorizer = TextVectorization(max_tokens=max_tokens,
                               standardize=None,
                               ngrams=(1, 2),
                               output_sequence_length=max_len)
vectorizer.adapt(X)

2022-02-02 16:02:53.745097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [130]:
y = y.reshape((-1,1))

In [131]:
print(X.shape)
print(y.shape)

(7613,)
(7613, 1)


In [132]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [133]:
np.save('data/x_train.npy', x_train)
np.save('data/y_train.npy', y_train)
np.save('data/x_test.npy', x_test)
np.save('data/y_test.npy', y_test)

In [134]:
sequences_input = Input(shape=(1), dtype='string')

vectorizer_sequence = vectorizer(sequences_input)

embedding_layer = Embedding(num_tokens,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            trainable=False)
embedded_sequences = embedding_layer(vectorizer_sequence)

x = Dense(200, activation="relu")(embedded_sequences)
x = Dropout(0.2)(x)
x = Dense(80, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(5, activation="relu")(x)
x = Dropout(0.2)(x)

preds = Dense(units=1,
              kernel_initializer='uniform',
              activation='sigmoid')(x)
model = Model(sequences_input, preds)

model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_14 (Text  (None, 68)               0         
 Vectorization)                                                  
                                                                 
 embedding_14 (Embedding)    (None, 68, 300)           15000600  
                                                                 
 dense_65 (Dense)            (None, 68, 200)           60200     
                                                                 
 dropout_51 (Dropout)        (None, 68, 200)           0         
                                                                 
 dense_66 (Dense)            (None, 68, 80)            16080     
                                                          

In [135]:
n_samples = x_train.shape[0]
model_n = 5

history = model.fit(x_train,
                    y_train,
                    validation_split=0.2,
                    batch_size=int(n_samples*0.1),
                    epochs=200)

model.save(f'models/model{model_n}.tf')

with open(f'models/history{model_n}.json', 'w') as f:
    json.dump(history.history, f)

Epoch 1/200


2022-02-02 16:02:55.385735: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5608

2022-02-02 16:02:56.476630: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 2s 108ms/step - loss: 0.6923 - accuracy: 0.5608 - val_loss: 0.6902 - val_accuracy: 0.5760
Epoch 2/200
9/9 [==============================] - 1s 68ms/step - loss: 0.6901 - accuracy: 0.5642 - val_loss: 0.6873 - val_accuracy: 0.5838
Epoch 3/200
9/9 [==============================] - 1s 67ms/step - loss: 0.6878 - accuracy: 0.5685 - val_loss: 0.6835 - val_accuracy: 0.5860
Epoch 4/200
9/9 [==============================] - 1s 65ms/step - loss: 0.6853 - accuracy: 0.5709 - val_loss: 0.6809 - val_accuracy: 0.5878
Epoch 5/200
9/9 [==============================] - 1s 65ms/step - loss: 0.6829 - accuracy: 0.5714 - val_loss: 0.6785 - val_accuracy: 0.5882
Epoch 6/200
9/9 [==============================] - 1s 65ms/step - loss: 0.6810 - accuracy: 0.5717 - val_loss: 0.6758 - val_accuracy: 0.5885
Epoch 7/200
9/9 [==============================] - 1s 66ms/step - loss: 0.6795 - accuracy: 0.5726 - val_loss: 0.6749 - val_accuracy: 0.5891
Epoch 8/200
9/9 [==============

2022-02-02 16:05:00.316358: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: models/model5.tf/assets
